In [16]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from tensorflow.keras.models import Sequential, Model, model_from_json
from skimage.transform import resize

In [17]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
#     print('Load file {} ...'.format(filename))
#     ds = pydicom.dcmread(filename)       
#     img = ds.pixel_array
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    
    print("Study Description: {0}".format(ds.StudyDescription))

    print("Modality: {0}".format(ds.Modality))
    if ds.Modality != "DX":
        print(">> Warning! Modality must be DX. Can't make a prediction.")
    print("Patient Sex: {0}".format(ds.PatientSex))
    if ds.PatientSex not in ['M', 'F']:
        print(">> Warning! Unsupported Patient_Sex value in DICOM file.")
    print("Body Part Examined: {0}".format(ds.BodyPartExamined))
    if ds.BodyPartExamined != 'CHEST':
        print(">> Warning! Unsupported value for Body_Part_Examined field. Can't make a prediction.")
    print("Patient Position: {0}".format(ds.PatientPosition))
    if ds.PatientPosition not in ['AP', 'PA']:
        print(">> Warning! Unsupported value for Patient_Position field. Can't make a prediction.")
    
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    proc_img = ((img/255)-img_mean)/img_std
    proc_img = resize(proc_img, img_size)
    return proc_img
    # todo
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        json_savedModel= json_file.read() 
        
    model = model_from_json(json_savedModel)
    model.load_weights(weight_path)
    # todo
    
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred = model.predict(img)
    if pred>thresh:
        prediction = 'Pneumonia Positive'
    else:
        prediction = 'Pneumonia Negative'
    
    # todo    
    
    return prediction 

In [18]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
'''
model_path = "my_model.json" #path to saved model
weight_path = "xray_class_my_model.best.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std =  1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.6 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)'''

'\nmodel_path = "my_model.json" #path to saved model\nweight_path = "xray_class_my_model.best.hdf5" #path to saved best weights\n\nIMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16\nimg_mean = 0 # loads the mean image value they used during training preprocessing\nimg_std =  1 # loads the std dev image value they used during training preprocessing\n\nmy_model = load_model(model_path, weight_path) #loads model\nthresh = 0.6 #loads the threshold they chose for model classification \n\n# use the .dcm files to test your prediction\nfor i in test_dicoms:\n    \n    img = np.array([])\n    img = check_dicom(i)\n    \n    if img is None:\n        continue\n        \n    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)\n    pred = predict_image(my_model,img_proc,thresh)\n    print(pred)'

In [19]:
model_path = "my_model.json" 
weight_path = 'out/xray_class_my_model.best.hdf5'
my_model = load_model(model_path, weight_path) #loads model
IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std =  1 # loads the std dev image value they used during training preprocessing
img = check_dicom('test1.dcm')
threshold = 0.6
img_proc = preprocess_image(img,img_mean, img_std, IMG_SIZE)
pred = predict_image(my_model, img_proc, threshold)
print('Pred: ', pred, '\n')


Load file test1.dcm ...
Study Description: No Finding
Modality: DX
Patient Sex: M
Body Part Examined: CHEST
Patient Position: PA
Pred:  Pneumonia Positive 



In [20]:
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    threshold = 0.6
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    
    #plt.imshow(img, cmap = 'bone')
    
    pred = predict_image(my_model,img_proc,threshold)
    print('Pred: ', pred, '\n')

Load file test1.dcm ...
Study Description: No Finding
Modality: DX
Patient Sex: M
Body Part Examined: CHEST
Patient Position: PA
Pred:  Pneumonia Positive 

Load file test2.dcm ...
Study Description: Cardiomegaly
Modality: DX
Patient Sex: M
Body Part Examined: CHEST
Patient Position: AP
Pred:  Pneumonia Positive 

Load file test3.dcm ...
Study Description: Effusion
Modality: DX
Patient Sex: M
Body Part Examined: CHEST
Patient Position: AP
Pred:  Pneumonia Positive 

Load file test4.dcm ...
Study Description: No Finding
Modality: DX
Patient Sex: M
Body Part Examined: RIBCAGE
>> Warning! Unsupported value for Body_Part_Examined field. Can't make a prediction.
Patient Position: PA
Pred:  Pneumonia Positive 

Load file test5.dcm ...
Study Description: No Finding
Modality: CT
>> Warning! Modality must be DX. Can't make a prediction.
Patient Sex: M
Body Part Examined: CHEST
Patient Position: PA
Pred:  Pneumonia Positive 

Load file test6.dcm ...
Study Description: No Finding
Modality: DX
Pat